In [12]:
# Import modules 
from scripts.dataset import Dataset
from scripts.classification_model import Classification_model
from scripts.classification_model import Hyper_params,Lr,Lr_reduce_on_plateau,Early_stop,Learning_params
from scripts.model_evaluator import Model_evaluator
from scripts import classifier_utils
from time import gmtime, strftime
from random import uniform
import sys
import os
import tensorflow as tf
from keras import backend as K
import keras.backend.tensorflow_backend as KTF
import numpy as np


def get_session(gpu_fraction=0.8):
    'Allocate gpu_fraction its memory for computattion'

    num_threads = os.environ.get('OMP_NUM_THREADS')
    gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=gpu_fraction)

    if num_threads:
        return tf.Session(config=tf.ConfigProto(
            gpu_options=gpu_options, intra_op_parallelism_threads=num_threads))
    else:
        return tf.Session(config=tf.ConfigProto(gpu_options=gpu_options))

KTF.set_session(get_session())
tf.GraphKeys.VARIABLES = tf.GraphKeys.GLOBAL_VARIABLES
cur_time = strftime("%Y-%m-%d %H:%M:%S", gmtime())

# This code supports tensorflow backend only
assert K.image_dim_ordering() == 'tf'

In [13]:
test_dir = 'test'
test_file = 'test_dataset.txt'
CLASS_NAMES = ['1521','1703','1707','1729','1751','1755','1837','2037','2525','2526']
PRETRAINED_MODEL = 'VGG19' #TODO: check using Resnet 50

In [14]:
# Copy images to test dir 

test_images = classifier_utils.load_test_images(test_file, test_dir,PRETRAINED_MODEL)
# Predict images and save


Copying images to test dir
pre-processing test/images dir
Found 4674 images belonging to 1 classes.


In [1]:
best_hyper_params = Hyper_params(lr=Lr(init=0.00102382988483,decay=0.0), optimization_method='Adam',
                           momentum=0.9, weight_decay=0.000173410239311, dropout_rate=0.491165771543,
                           freeze_layer_id=21, architecture_head_type='no_layers',
                           is_use_batch_normalization=True, nb_filters_conv_1_1_first=128,
                           nb_filters_conv_1_1_second=32, fc_layer_size_1=128, fc_layer_size_2=64)

prod_weights_dir = 'best_model'
prod_summary_dir = prod_weights_dir
prod_model = Classification_model(10, prod_weights_dir, prod_weights_dir, PRETRAINED_MODEL,False)
prod_model.build_model(input_shape=test_images[0,:,:,:].shape,
                       hyper_params=best_hyper_params)

prod_model.model.load_weights('best_model/production_weights.h5')#os.path.join(prod_weights_dir,'production_weights.h5'))
predictions = prod_model.model.predict(test_images,verbose=1)
out_classes = np.argmax(predictions, axis=1)

In [ ]:
# Generate output
classifier_utils.generate_output_file(test_file, 'results_test.txt', out_classes, CLASS_NAMES)